In [60]:
%load_ext autoreload
%autoreload 2
%matplotlib inline 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import os, sys
sys.path.append('../')
import ama

In [84]:
from keras.optimizers import Adam

In [71]:
Vgg = ama.vgg.Vgg
TrainBatch = ama.trainbatch.TrainBatch

In [63]:
path= '../data/'
cv_version = 1
cv_version = str(cv_version)
batch_size = 128
img_size = (64,64)

In [74]:
vgg = Vgg(input_shape=(3,)+img_size, dropout=0)
vgg.load_weights('../data/weights/atc_1.hk', loaded_dropout=0.25)

In [75]:
valgen = TrainBatch(path+'val-jpg'+cv_version+'/', path+'train_v2.csv', batch_size=batch_size, img_size=img_size)

Found 6000 images belonging to 1 classes.


In [76]:
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(rotation_range=5,
        horizontal_flip=True, vertical_flip=True)

traingen = TrainBatch(path+'train-jpg'+cv_version+'/', path+'train_v2.csv', batch_size=batch_size, img_size=img_size,
                      imagegen=gen)

Found 34479 images belonging to 1 classes.


In [77]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from ama.persistenthistory import PersistentHistory

earlystop = EarlyStopping(patience=4)
history = PersistentHistory()

In [81]:
def get_saver(fp):
    return ModelCheckpoint(filepath, verbose=1, save_best_only=True, save_weights_only=True)

In [82]:
filepath = '../data/weights/atc'+cv_version+'_less_dropout_best_lr1.hk'
saver = get_saver(filepath)

In [85]:
vgg.model.compile(optimizer=Adam(lr=0.001),loss='binary_crossentropy', metrics=['accuracy'])
vgg.model.fit_generator(traingen, samples_per_epoch=traingen.nb_sample, nb_epoch=20,
                        validation_data=valgen, nb_val_samples=valgen.nb_sample,
                        callbacks=[history, saver, earlystop])

Epoch 1/20
34479/34479 [==============================] - 396s - loss: 0.0963 - acc: 0.9622 - val_loss: 0.1022 - val_acc: 0.9608
Epoch 2/20
34479/34479 [==============================] - 180s - loss: 0.0946 - acc: 0.9628 - val_loss: 0.1037 - val_acc: 0.9610
Epoch 3/20
34479/34479 [==============================] - 180s - loss: 0.0923 - acc: 0.9638 - val_loss: 0.1042 - val_acc: 0.9612
Epoch 4/20
34479/34479 [==============================] - 181s - loss: 0.0908 - acc: 0.9641 - val_loss: 0.1030 - val_acc: 0.9613
Epoch 5/20
34479/34479 [==============================] - 181s - loss: 0.0889 - acc: 0.9648 - val_loss: 0.1049 - val_acc: 0.9607
Epoch 6/20
34479/34479 [==============================] - 180s - loss: 0.0870 - acc: 0.9656 - val_loss: 0.1054 - val_acc: 0.9602


I like that the model is training and overfitting a bit. I think this means we need more augmentation basically. I'm also thinking about removing / extending the patience on the early stopping, though. Let's first try a lower LR.

In [87]:
vgg.model.compile(optimizer=Adam(lr=0.0001),loss='binary_crossentropy', metrics=['accuracy'])
earlystop = EarlyStopping(patience=5)
vgg.model.fit_generator(traingen, samples_per_epoch=traingen.nb_sample, nb_epoch=10,
                        validation_data=valgen, nb_val_samples=valgen.nb_sample,
                        callbacks=[history, saver, earlystop])

Epoch 1/10
34479/34479 [==============================] - 182s - loss: 0.0815 - acc: 0.9677 - val_loss: 0.0985 - val_acc: 0.9620
Epoch 2/10
34479/34479 [==============================] - 180s - loss: 0.0794 - acc: 0.9685 - val_loss: 0.0997 - val_acc: 0.9621
Epoch 3/10
34479/34479 [==============================] - 180s - loss: 0.0785 - acc: 0.9689 - val_loss: 0.0998 - val_acc: 0.9623
Epoch 4/10
34479/34479 [==============================] - 181s - loss: 0.0778 - acc: 0.9693 - val_loss: 0.1004 - val_acc: 0.9620
Epoch 5/10
34479/34479 [==============================] - 181s - loss: 0.0771 - acc: 0.9694 - val_loss: 0.1000 - val_acc: 0.9622
Epoch 6/10
34479/34479 [==============================] - 181s - loss: 0.0765 - acc: 0.9697 - val_loss: 0.0996 - val_acc: 0.9622
Epoch 7/10
34479/34479 [==============================] - 181s - loss: 0.0761 - acc: 0.9698 - val_loss: 0.1004 - val_acc: 0.9619


Looks like we crept below 0.1, which is good. The real baddy boys @ 0.93 on the LB are pushing 0.08 loss though.

In [89]:
vgg.model.load_weights(filepath)
filepath = '../data/weights/atc'+cv_version+'_less_dropout_best_lr3.hk'
saver = get_saver(filepath)

In [90]:
vgg.model.compile(optimizer=Adam(lr=0.00001),loss='binary_crossentropy', metrics=['accuracy'])
vgg.model.fit_generator(traingen, samples_per_epoch=traingen.nb_sample, nb_epoch=10,
                        validation_data=valgen, nb_val_samples=valgen.nb_sample,
                        callbacks=[history, saver, earlystop])

Epoch 1/10
34479/34479 [==============================] - 179s - loss: 0.0791 - acc: 0.9688 - val_loss: 0.0976 - val_acc: 0.9623
Epoch 2/10
34479/34479 [==============================] - 180s - loss: 0.0795 - acc: 0.9686 - val_loss: 0.0976 - val_acc: 0.9622
Epoch 3/10
34479/34479 [==============================] - 183s - loss: 0.0789 - acc: 0.9689 - val_loss: 0.0975 - val_acc: 0.9623
Epoch 4/10
34479/34479 [==============================] - 180s - loss: 0.0785 - acc: 0.9689 - val_loss: 0.0982 - val_acc: 0.9622
Epoch 5/10
34479/34479 [==============================] - 182s - loss: 0.0784 - acc: 0.9689 - val_loss: 0.0974 - val_acc: 0.9622
Epoch 6/10
34479/34479 [==============================] - 181s - loss: 0.0786 - acc: 0.9688 - val_loss: 0.0988 - val_acc: 0.9623
Epoch 7/10
34479/34479 [==============================] - 181s - loss: 0.0781 - acc: 0.9691 - val_loss: 0.0992 - val_acc: 0.9623
Epoch 8/10
34479/34479 [==============================] - 181s - loss: 0.0784 - acc: 0.9690 - val

We're overfitting now. Let's try more data augmentation, and after that we can crank up the dropout a bit if necessary. I want to try to avoid adding too much "noise" to the data (ie through augmentations that mangle the data too much - think about whats happening to the image - or through too much dropout) as this will stop us learning enough to get our val loss down.

In [91]:
vgg.model.load_weights(filepath)
filepath = '../data/weights/atc'+cv_version+'_less_dropout_more_aug_best_lr3.hk'
saver = get_saver(filepath)

In [92]:
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(rotation_range=10, shear_range=10,
        horizontal_flip=True, vertical_flip=True)

traingen = TrainBatch(path+'train-jpg'+cv_version+'/', path+'train_v2.csv', batch_size=batch_size, img_size=img_size,
                      imagegen=gen)

Found 34479 images belonging to 1 classes.


In [93]:
vgg.model.compile(optimizer=Adam(lr=0.001),loss='binary_crossentropy', metrics=['accuracy'])
vgg.model.fit_generator(traingen, samples_per_epoch=traingen.nb_sample, nb_epoch=10,
                        validation_data=valgen, nb_val_samples=valgen.nb_sample,
                        callbacks=[history, saver, earlystop])

Epoch 1/10
34479/34479 [==============================] - 181s - loss: 0.1254 - acc: 0.9530 - val_loss: 0.1058 - val_acc: 0.9595
Epoch 2/10
34479/34479 [==============================] - 184s - loss: 0.1200 - acc: 0.9550 - val_loss: 0.1035 - val_acc: 0.9604
Epoch 3/10
34479/34479 [==============================] - 183s - loss: 0.1199 - acc: 0.9547 - val_loss: 0.1031 - val_acc: 0.9608
Epoch 4/10
34479/34479 [==============================] - 185s - loss: 0.1190 - acc: 0.9551 - val_loss: 0.1024 - val_acc: 0.9610
Epoch 5/10
11392/34479 [========>.....................] - ETA: 111s - loss: 0.1206 - acc: 0.9546

KeyboardInterrupt: 

Ok, it really didn't like that - was it cause I cranked the LR up too high or something? Let's try again the same setup but with LR/10 (after reloading the weights from before). Otherwise, we can get rid of some of the augmentation since it was underfitting a lot.

In [97]:
vgg.model.load_weights('../data/weights/atc'+cv_version+'_less_dropout_best_lr3.hk')
filepath = '../data/weights/atc'+cv_version+'_less_dropout_more_aug_best_lr3.hk'
saver = get_saver(filepath)

Before we go any further, I'm just gonna check this is loading the weights right - aka that the val loss should be 0.0974

In [99]:
vgg.model.evaluate_generator(valgen, valgen.nb_sample)

[0.09742651663223903, 0.96219607512156169]

Sweet, let's crack on

In [100]:
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(rotation_range=10, shear_range=10,
        horizontal_flip=True, vertical_flip=True)

traingen = TrainBatch(path+'train-jpg'+cv_version+'/', path+'train_v2.csv', batch_size=batch_size, img_size=img_size,
                      imagegen=gen)

Found 34479 images belonging to 1 classes.


In [101]:
vgg.model.compile(optimizer=Adam(lr=0.0001),loss='binary_crossentropy', metrics=['accuracy'])
vgg.model.fit_generator(traingen, samples_per_epoch=traingen.nb_sample, nb_epoch=10,
                        validation_data=valgen, nb_val_samples=valgen.nb_sample,
                        callbacks=[history, saver, earlystop])

Epoch 1/10
34479/34479 [==============================] - 182s - loss: 0.1292 - acc: 0.9528 - val_loss: 0.1056 - val_acc: 0.9598
Epoch 2/10
34479/34479 [==============================] - 182s - loss: 0.1217 - acc: 0.9547 - val_loss: 0.1020 - val_acc: 0.9610
Epoch 3/10
34479/34479 [==============================] - 185s - loss: 0.1192 - acc: 0.9556 - val_loss: 0.1017 - val_acc: 0.9613
Epoch 4/10
34479/34479 [==============================] - 182s - loss: 0.1181 - acc: 0.9557 - val_loss: 0.0994 - val_acc: 0.9614
Epoch 5/10
34479/34479 [==============================] - 181s - loss: 0.1164 - acc: 0.9566 - val_loss: 0.0997 - val_acc: 0.9616
Epoch 6/10
34479/34479 [==============================] - 182s - loss: 0.1166 - acc: 0.9564 - val_loss: 0.0995 - val_acc: 0.9617
Epoch 7/10
34479/34479 [==============================] - 183s - loss: 0.1157 - acc: 0.9568 - val_loss: 0.0994 - val_acc: 0.9614
Epoch 8/10
34479/34479 [==============================] - 182s - loss: 0.1147 - acc: 0.9570 - val

Ok, I'm not convinced. We're underfitting again but I think maybe the noise is too much. I'm gonna lower the aug down to 5 deg on rotation & shear.

In [102]:
vgg.model.load_weights('../data/weights/atc'+cv_version+'_less_dropout_best_lr3.hk')
filepath = '../data/weights/atc'+cv_version+'_less_dropout_more_aug_best_lr3.hk'
saver = get_saver(filepath)

In [103]:
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(rotation_range=5, shear_range=5,
        horizontal_flip=True, vertical_flip=True)

traingen = TrainBatch(path+'train-jpg'+cv_version+'/', path+'train_v2.csv', batch_size=batch_size, img_size=img_size,
                      imagegen=gen)

Found 34479 images belonging to 1 classes.


In [104]:
vgg.model.compile(optimizer=Adam(lr=0.0001),loss='binary_crossentropy', metrics=['accuracy'])
vgg.model.fit_generator(traingen, samples_per_epoch=traingen.nb_sample, nb_epoch=10,
                        validation_data=valgen, nb_val_samples=valgen.nb_sample,
                        callbacks=[history, saver, earlystop])

Epoch 1/10
34479/34479 [==============================] - 182s - loss: 0.1331 - acc: 0.9516 - val_loss: 0.1073 - val_acc: 0.9598
Epoch 2/10
34479/34479 [==============================] - 181s - loss: 0.1253 - acc: 0.9535 - val_loss: 0.1026 - val_acc: 0.9610
Epoch 3/10
34479/34479 [==============================] - 181s - loss: 0.1221 - acc: 0.9545 - val_loss: 0.1013 - val_acc: 0.9610
Epoch 4/10
34479/34479 [==============================] - 183s - loss: 0.1206 - acc: 0.9550 - val_loss: 0.1005 - val_acc: 0.9610
Epoch 5/10
34479/34479 [==============================] - 183s - loss: 0.1203 - acc: 0.9551 - val_loss: 0.1004 - val_acc: 0.9612
Epoch 6/10
34479/34479 [==============================] - 183s - loss: 0.1190 - acc: 0.9556 - val_loss: 0.0998 - val_acc: 0.9615
Epoch 7/10
34479/34479 [==============================] - 185s - loss: 0.1189 - acc: 0.9554 - val_loss: 0.0995 - val_acc: 0.9615
Epoch 8/10
34479/34479 [==============================] - 187s - loss: 0.1177 - acc: 0.9560 - val

So slow to get down. I'm gonna try higher for a long time, do some other jobs, and come back to check it out later.

In [106]:
vgg.model.compile(optimizer=Adam(lr=0.001),loss='binary_crossentropy', metrics=['accuracy'])
vgg.model.fit_generator(traingen, samples_per_epoch=traingen.nb_sample, nb_epoch=30,
                        validation_data=valgen, nb_val_samples=valgen.nb_sample,
                        callbacks=[history, saver, earlystop])

Epoch 1/30
34479/34479 [==============================] - 179s - loss: 0.1229 - acc: 0.9538 - val_loss: 0.1044 - val_acc: 0.9594
Epoch 2/30
34479/34479 [==============================] - 181s - loss: 0.1226 - acc: 0.9538 - val_loss: 0.1038 - val_acc: 0.9595
Epoch 3/30
34479/34479 [==============================] - 181s - loss: 0.1215 - acc: 0.9541 - val_loss: 0.1029 - val_acc: 0.9595
Epoch 4/30
34479/34479 [==============================] - 180s - loss: 0.1211 - acc: 0.9543 - val_loss: 0.1008 - val_acc: 0.9607
Epoch 5/30
34479/34479 [==============================] - 180s - loss: 0.1202 - acc: 0.9545 - val_loss: 0.1032 - val_acc: 0.9599
Epoch 6/30
34479/34479 [==============================] - 180s - loss: 0.1186 - acc: 0.9552 - val_loss: 0.1018 - val_acc: 0.9606
Epoch 7/30
34479/34479 [==============================] - 181s - loss: 0.1193 - acc: 0.9547 - val_loss: 0.1031 - val_acc: 0.9593
Epoch 8/30
34479/34479 [==============================] - 180s - loss: 0.1193 - acc: 0.9548 - val

In [107]:
vgg.model.compile(optimizer=Adam(lr=0.001),loss='binary_crossentropy', metrics=['accuracy'])
vgg.model.fit_generator(traingen, samples_per_epoch=traingen.nb_sample, nb_epoch=30,
                        validation_data=valgen, nb_val_samples=valgen.nb_sample,
                        callbacks=[history, saver, earlystop])

Epoch 1/30
34479/34479 [==============================] - 179s - loss: 0.1180 - acc: 0.9556 - val_loss: 0.1025 - val_acc: 0.9601
Epoch 2/30
34479/34479 [==============================] - 181s - loss: 0.1193 - acc: 0.9551 - val_loss: 0.2629 - val_acc: 0.9189
Epoch 3/30
34479/34479 [==============================] - 181s - loss: 0.1332 - acc: 0.9504 - val_loss: 0.1073 - val_acc: 0.9595
Epoch 4/30
34479/34479 [==============================] - 180s - loss: 0.1210 - acc: 0.9546 - val_loss: 0.1033 - val_acc: 0.9600
Epoch 5/30
34479/34479 [==============================] - 180s - loss: 0.1179 - acc: 0.9557 - val_loss: 0.1018 - val_acc: 0.9605
Epoch 6/30
34479/34479 [==============================] - 180s - loss: 0.1162 - acc: 0.9561 - val_loss: 0.1011 - val_acc: 0.9606
Epoch 7/30
34479/34479 [==============================] - 180s - loss: 0.1152 - acc: 0.9565 - val_loss: 0.1026 - val_acc: 0.9603
Epoch 8/30
 6016/34479 [====>.........................] - ETA: 136s - loss: 0.1170 - acc: 0.9555

KeyboardInterrupt: 

Really not convinced by this. Gonna try increasing dropout a bit and training on the old aug techniques.

In [108]:
vgg = Vgg(input_shape=(3,)+img_size, dropout=0.1)
vgg.load_weights('../data/weights/atc'+cv_version+'_less_dropout_best_lr3.hk', loaded_dropout=0)

In [109]:
filepath = '../data/weights/atc'+cv_version+'_slightly_less_dropout_best_lr3.hk'
saver = get_saver(filepath)

In [110]:
valgen = TrainBatch(path+'val-jpg'+cv_version+'/', path+'train_v2.csv', batch_size=batch_size, img_size=img_size)

Found 6000 images belonging to 1 classes.


In [111]:
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(rotation_range=5,
        horizontal_flip=True, vertical_flip=True)

traingen = TrainBatch(path+'train-jpg'+cv_version+'/', path+'train_v2.csv', batch_size=batch_size, img_size=img_size,
                      imagegen=gen)

Found 34479 images belonging to 1 classes.


In [112]:
vgg.model.compile(optimizer=Adam(lr=0.001),loss='binary_crossentropy', metrics=['accuracy'])
vgg.model.fit_generator(traingen, samples_per_epoch=traingen.nb_sample, nb_epoch=20,
                        validation_data=valgen, nb_val_samples=valgen.nb_sample,
                        callbacks=[history, saver, earlystop])

Epoch 1/20
34479/34479 [==============================] - 179s - loss: 0.0863 - acc: 0.9658 - val_loss: 0.1074 - val_acc: 0.9596
Epoch 2/20
34479/34479 [==============================] - 181s - loss: 0.0846 - acc: 0.9665 - val_loss: 0.1052 - val_acc: 0.9608
Epoch 3/20
34479/34479 [==============================] - 183s - loss: 0.0836 - acc: 0.9668 - val_loss: 0.1051 - val_acc: 0.9605
Epoch 4/20
34479/34479 [==============================] - 181s - loss: 0.0827 - acc: 0.9673 - val_loss: 0.1067 - val_acc: 0.9599
Epoch 5/20
34479/34479 [==============================] - 180s - loss: 0.0809 - acc: 0.9681 - val_loss: 0.1074 - val_acc: 0.9600
Epoch 6/20
34479/34479 [==============================] - 180s - loss: 0.0804 - acc: 0.9680 - val_loss: 0.1100 - val_acc: 0.9603
Epoch 7/20
34479/34479 [==============================] - 181s - loss: 0.0791 - acc: 0.9687 - val_loss: 0.1080 - val_acc: 0.9601
Epoch 8/20
34479/34479 [==============================] - 180s - loss: 0.0765 - acc: 0.9693 - val